**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
from tqdm import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

## тексты

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
1693,"Управление Директора национальной разведки США (ODNI)отказалось поддержать версию, ранее выдвинутую ЦРУ, о том, что Россия якобы проводила хакерские атаки, чтобы помочь республиканцу Дональду Трам...",politics
2746,"АО «Гознак» будет изготавливать денежные купюры с прозрачными участками для повышения безопасности от подделки. Отмечается, что в данных Роспатента, информация о технологиях Гознака уже имеется.\n...",science
3056,"В четверг, 4 августа 2010 года Хакасское УФАС России возбудило дело в отношении открытого акционерного общества <object>МРСК Сибири</object> по признакам нарушения части 1 статьи 10 Закона о защит...",incident
1273,Galaxy Note 7 — всё: Samsung принудительно заблокирует оставшиеся смартфоны\n\n10 декабря 2016 в 9:59\n\nLenta.ru\n\nКомпания Samsung принудительно отключит оставшиеся в обращении Galaxy Note 7. О...,tech
374,"Аутизм связан с наследственными изменениями в строении мозга Родственники\nлюдей, страдающих аутизмом, имеют сходные отклонения в строении мозга.\nОб этом сообщили сразу две научные группы на 35-й...",health


In [3]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [4]:
# количество категорий
data['tag'].drop_duplicates().count()

13

In [5]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

## очистка и токенизация 

In [6]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split(' ') if (len(w)>1) ]

In [7]:
data['ctext'] = data['text'].progress_apply(string_normalizer).progress_apply( tokenize )

100%|██████████| 3196/3196 [00:00<00:00, 34587.38it/s]


In [8]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 83094 слов


In [9]:
data.sample(3)

,text,tag,ctext
2599,"Вымершие пернатые обитали на островах\n\nВ Красную книгу были добавлены 740 новых видов птиц, из них 13 профессионалы признали вымершими.\n\nПо утверждению исследователей, виды вымерших птиц обита...",science,"[вымершие, пернатые, обитали, на, островах, красную, книгу, были, добавлены, новых, видов, птиц, из, них, профессионалы, признали, вымершими, по, утверждению, исследователей, виды, вымерших, птиц,..."
2926,Китайцы выпустят клона Mercedes-Benz Gelаndewagen\n\nКитайцы выпустят новый внедорожник под названием Beijing BJ80. Он является по сути клоном известного автомобиля Mercedes-Benz G-Class.\n\nФото:...,auto,"[китайцы, выпустят, клона, mercedes-benz, gelаndewagen, китайцы, выпустят, новый, внедорожник, под, названием, beijing, bj, он, является, по, сути, клоном, известного, автомобиля, mercedes-benz, g..."
63,"""Синдром большого пальца"": новая болезнь, связанная с технологиями\nБританские врачи предупреждают о появлении нового недуга, который они\nназывают ""синдромом большого пальца"" или же ""палец iPod""....",health,"[синдром, большого, пальца, новая, болезнь, связанная, технологиями, британские, врачи, предупреждают, появлении, нового, недуга, который, они, называют, синдромом, большого, пальца, или, же, пале..."


## стеминг

In [10]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): 
    return [ stemmer.stem(w) for w in s if w not in stopwords ]

In [11]:
data['ctext'] = data['ctext'].progress_apply( stem )

100%|██████████| 3196/3196 [00:14<00:00, 214.71it/s]


In [12]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc1.txt',index=False,header=False)

словарь 36381 слов


In [13]:
data.sample(3)

,text,tag,ctext
401,"Французы провели первую в мире пересадку лица 36-летняя француженка стала\nпервой пациенткой в мире, получившей новое лицо в ходе пятичасовой\nоперации. Лицо пациентки было изуродовано в результат...",health,"[француз, провел, перв, мир, пересадк, лиц, француженк, стал, перв, пациентк, мир, получ, нов, лиц, ход, пятичасов, операц, лиц, пациентк, изуродова, результат, нападен, собак, хирург, замен, лиш,..."
730,"Жители Костюковки о новых названиях своих улиц: ""Никто нашего мнения не спрашивал""\n\n8 декабря 2016 в 15:36\n\nАлена Германович, Naviny.by\n\nСердитые смайлики появляются под новостью в социальны...",social,"[жител, костюковк, нов, назван, сво, улиц, никт, наш, мнен, спрашива, декабр, ал, германович, naviny, by, сердит, смайлик, появля, новост, социальн, сет, переименован, улиц, костюковк, бывш, рабоч..."
229,Алла Пугачева развелась с Киркоровым и затаилась в Нижнем Тагиле Алла\nПугачева и Филипп Киркоров официально заявили о своем разводе. Правда\nсделали это через посредника - певицу Лолиту Милявскую...,culture,"[алл, пугачев, развел, киркоров, зата, нижн, тагил, алл, пугачев, филипп, киркор, официальн, заяв, сво, развод, правд, сдела, эт, посредник, певиц, лолит, милявск, котор, озвуч, новост, наканун, в..."


## лемматизация

In [14]:
# лемматизация и очистка с помощью пакета морфологического анализа

from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

# NOUN (существительное), VERB (глагол), ADJF (прилагательное)
def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''

def text_normalizer_tokenize(s):
    return [ word_normalizer(w) for w in s.split(' ') if len(w)>1 ]

In [15]:
data['ctext'] = data['text']\
                   .progress_apply(string_normalizer)\
                   .progress_apply( text_normalizer_tokenize )

100%|██████████| 3196/3196 [01:22<00:00, 38.93it/s]


In [16]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc2.txt',index=False,header=False)

словарь 22673 слов


In [17]:
data.sample(3)

,text,tag,ctext
2382,"По данным пресс-службы столичной автоинспекции, авария произошла на пересечение улицы Мосфильмовская с Университетским проспектом. Девочек, которым 8 и 9 лет, направили в больницу на осмотр. По пр...",incident,"[, данные, пресс-служба, столичный, автоинспекция, авария, произойти, , пересечение, улица, , университетский, проспект, девочка, который, год, направить, больница, , осмотр, , предварительный, да..."
2076,"Sport Arena выделяет ключевые моменты матча на Стадио Олимпико, завершившегося минимальной победой хозяев.\n\nРома – Милан: как это было\n\nЛучано Спаллетти был честен и прав\n\nМатематически преи...",sport,"[, , выделять, ключевой, момент, матч, , , , , минимальный, победа, хозяин, рома, милан, , , быть, , , быть, , право, , преимущество, белый, , чёрный, , сказать, расизм, шахматы, , , , , статистик..."
793,"Инсталляция появилась на кольце на Варшавском шоссе, сообщает BrestCITY.com . \r\nСветящийся фонтан установлен в месте, где ранее планировалось разместить въездную стелу .\r\n\r\nФотофакт. В Брест...",culture,"[инсталляция, появиться, , кольцо, , варшавский, шоссе, сообщать, , , светящийся, фонтан, , место, , , планироваться, , въездной, стела, , брест, новый, год, установить, большой, светящийся, фонтан]"
